In [106]:
#import
import os
import numpy as np
import scipy.io as sio
import scipy as sp
import pandas as pd
import seaborn as sns

np.random.seed(0)

### Replication from White Matter Network Architecture Guides Direct Electrical Stimulation through Optimal State Transitions
Direct electrical brain stimulation is useful treatment for some neurological disorders (most famously, Parkinson's). However, we still don't really have a good way of predicting how stimulation at single region will influnce the rest of the brain. Here, we hypothesize that input from stimulation will spread linearly along white matter tracts to impact brain activity at distant regions. We use a dataset of combined iEEG, DWI, and stimulation data to simulate stimulation based on this hypothesis.

Data consist of one DWi structural adjacency matrix per dataset, and variable number of stimulation trials. Each stimulation trial includes the brain state before and after stimulation.

In [19]:
# load data
workdir = '/Users/stiso/Documents/Code/test_data/'
stim = ['R1170J_1.PS2_1_set1', 'R1173J.PS2_1_set1', 'R1170J_1.sham_PS2_1_set2']
sham = ['R1173J.sham_PS2_1_set1', 'R1170J_1.sham_PS2_1_set1', 'R1170J_1.sham_PS2_1_set2'];
dat = sio.loadmat(f'{workdir}Trajectory_Data.{subj[0]}.mat')
dat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Electrode_ROI', 'Post_Freq_State', 'Post_Stim_Prob', 'Pre_Freq_State', 'Pre_Stim_Prob', 'Stim_Amp', 'Stim_Duration', 'Stim_Freq', 'Stim_Loc_Idx', 'Struct_Adj'])

In [115]:
#set parameters
# these were the parameters were used in the the paper. they were chosen to minimuze error, 
# reults were validated in 2 additional parameter sets

# balance between minimizing energy or minimizing distance from target state
rho = .2  
# time to go from initial to target state
T = .7      
# the number of time points the code spits out: T * 1000 + 1
nTime = 701 
gamma = 4
# to try and simulate stimuluation, we're gonna weight the B matrix
B_mu = .0005 
B_sigma = .00005
scale = 250

#### Does it take more energy to get to post-stimulation state, or a  sham stimulation state?

In [ ]:
from network_control.utils import matrix_normalization
from network_control.energies import optimal_input

# intiialize final data strcutre
energies = pd.DataFrame(columns=['energy', 'condition', 'subject', 'trial', 'error'])

for i,s in enumerate(stim):
    # load in data
    stim_dat = sio.loadmat(f'{workdir}Trajectory_Data.{s}.mat')
    sham_dat = sio.loadmat(f'{workdir}Trajectory_Data.{sham[i]}.mat')
    
    # subject specific constants
    # number of stim trials
    nTrial = np.size(stim_dat['Post_Stim_Prob'])
    # number of nodes/regions in the atlas we are using
    nROI = np.size(stim_dat['Post_Freq_State'],1) 
    # number of bands
    nFreq = np.size(stim_dat['Post_Freq_State'],2)
    # these are the regions with contacts
    elec_idx = np.sum(stim_dat['Post_Freq_State'][:,:,0],0) != 0 
    ROI_idx = [not x for x in elec_idx]
    # number of contacts
    nElec = sum(elec_idx)
    # stim contacts
    stim_idx = [x[0] for x in stim_dat['Stim_Loc_Idx'][0][0]] 
    # which regions we want to constrain the state of
    S = np.eye(nROI)
    
    # scale A matrix (continuous)
    # this variable will be the same for both datasets
    A = stim_dat['Struct_Adj'] 
    matrix_normalization(A, c=gamma, version='continuous')
    
    # set B matrix - ultimate goal is to have the majority of input be at the stim elecs
    # first, we set small input everywhere
    B = np.eye(nROI) * np.random.normal(loc=B_mu, scale=B_sigma, size=(1, nROI)) 
    # then we add 0s to all the areas whos activity we know
    B[elec_idx,elec_idx] = 0 
    # then, we add big numbers to the stim elecs
    B[stim_idx,stim_idx] = np.eye(np.size(stim_idx))
    
    # get optimal input and trajectory for each trial
    for t in range(nTrial):
        # get states
        x0_stim = np.squeeze(stim_dat['Pre_Freq_State'][t,:,:])
        x0_sham = np.squeeze(stim_dat['Post_Freq_State'][t,:,:])
        
        xf_stim = np.squeeze(sham_dat['Pre_Freq_State'][t,:,:])
        xf_sham = np.squeeze(sham_dat['Post_Freq_State'][t,:,:])
        
        # add 1s to regions without elecs
        x0_stim[ROI_idx,:] = 1
        x0_sham[ROI_idx,:] = 1
        
        xf_stim[ROI_idx,:] = 1
        xf_sham[ROI_idx,:] = 1
        
        # concatenate across frequency bands
        u_stim = np.zeros((nROI,nTime,nFreq))
        u_sham = np.zeros((nROI,nTime,nFreq))
        err_stim = np.zeros((1,nFreq))
        err_sham = np.zeros((1,nFreq))
        for f in range(nFreq):
            _,curr_u_stim, curr_err_stim = optimal_input(A,T,B,x0_stim[:,f],xf_stim[:,f],rho,S)
            _,curr_u_sham, curr_err_sham = optimal_input(A,T,B,x0_sham[:,f],xf_sham[:,f],rho,S)
            
            u_stim[:,:,f] = curr_u_stim.T
            u_sham[:,:,f] = curr_u_sham.T
            err_stim[:,f] = curr_err_stim
            err_stim[:,f] = curr_err_stim
        
        # get summary of optimal input
        # we incorporated the B matrix into our input summary because of the weighting
        # we use the term energy to be consistent with other literature, but in some sense this is a different summary statistic
        eng_stim = np.mean(np.linalg.norm(B*u_stim))/1000
        eng_sham = np.mean(np.linalg.norm(B*u_sham))/1000
        # average err over frequencies
        err_stim = np.mean(err_stim)
        err_stim = np.mean(err_sham)
        
        # add to data frame
        curr_stim = pd.DataFrame(energy=eng_stim, condition='stim', subject=s, trial=t, error=err_stim)
        curr_sham = pd.DataFrame(energy=eng_stim, condition='stim', subject=s, trial=t, error=err_stim)
        energies = pd.concat([energies,curr_stim,curr_sham])
    
    

Normalizing A for a continuous-time system


In [103]:
np.shape(x0_sham)

(463, 8)

In [66]:
k = [x[0][0] for x in stim_dat['Stim_Loc_Idx']]
k[0]

array([435], dtype=uint16)